[Medium code](https://towardsdatascience.com/4-bit-quantization-with-gptq-36b0f4f02c34)

[The Bloke repo ](https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ)

[Load and use ](https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e)pre quantized model from HF

[AWQ vs GGUF vs GPTQ](https://medium.com/towards-data-science/which-quantization-method-is-right-for-you-gptq-vs-gguf-vs-awq-c4cd9d77d5be)

[GPTQ medium](https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e)

#Install libraries

In [ ]:
!pip install einops
# !pip install flash-attn --no-build-isolation
# !pip install optimum
!pip install accelerate
!pip install evaluate
# !pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes
# !pip install trl
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 782.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [ ]:
# AutoGPTQ library:
!pip install auto-gptq

# Optimum library:
!pip install --upgrade optimum

# Install latest transformers library from source:
!pip install --upgrade git+https://github.com/huggingface/transformers.git

# Install latest accelerate library:
# !pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-clhsaag4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-clhsaag4
  Resolved https://github.com/huggingface/transformers.git to commit 91d155ea92da372b319a79dd4eef69533ee15170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9045382 sha256=7ea91eb8b59c5afdc08b7ed8e

In [ ]:
!BUILD_CUDA_EXT=0 pip install -q auto-gptq transformers

# Load Libraries

In [ ]:
import random

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers import (
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoConfig
)

"""
To install a library, run the following command:
!pip install library_name
"""
import warnings
warnings.filterwarnings('ignore') # to avoid warnings
import os,nltk,evaluate

# from transformers import BertTokenizer,  AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from peft import PeftModel  # 0.5.0
import torch

"""
Pytorch Libraries
"""
from transformers import pipeline
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

import pandas as pd
from tqdm import tqdm


# Quantization of GPT2 - GPTQ

## Load Model

In [ ]:
# Define base model and output directory
model_id = "gpt2"
out_dir = model_id + "-GPTQ"

# Load quantize config, model and tokenizer
quantize_config = BaseQuantizeConfig(
    bits=2,
    group_size=128,
    damp_percent=0.01,
    desc_act=False,
)
# internlm7b_tokenizer
# internlm7b_model


internlm_model_gptq = AutoGPTQForCausalLM.from_pretrained(model_id, quantize_config, trust_remote_code=True)
tokenizer_model_gptq = AutoTokenizer.from_pretrained(model_id)

[C4 HF](https://huggingface.co/datasets/allenai/c4)

In [ ]:
# Load data and tokenize examples
n_samples = 1024
data = load_dataset("allenai/c4", data_files="en/c4-train.00001-of-01024.json.gz", split=f"train[:{n_samples*5}]")
tokenized_data = tokenizer_model_gptq("\n\n".join(data['text']), return_tensors='pt')

# Format tokenized examples
examples_ids = []
for _ in range(n_samples):
    i = random.randint(0, tokenized_data.input_ids.shape[1] - tokenizer_model_gptq.model_max_length - 1)
    j = i + tokenizer_model_gptq.model_max_length
    input_ids = tokenized_data.input_ids[:, i:j]
    attention_mask = torch.ones_like(input_ids)
    examples_ids.append({'input_ids': input_ids, 'attention_mask': attention_mask})

Token indices sequence length is longer than the specified maximum sequence length for this model (2441065 > 1024). Running this sequence through the model will result in indexing errors


##**Quantize with GPTQ**

In [ ]:

internlm_model_gptq.quantize(
    examples_ids,
    batch_size=1,
    use_triton=True,
)

# Save model and tokenizer
internlm_model_gptq.save_quantized(out_dir, use_safetensors=True)
tokenizer_model_gptq.save_pretrained(out_dir)

INFO - Start quantizing layer 1/12
INFO:auto_gptq.modeling._base:Start quantizing layer 1/12
INFO - Quantizing attn.c_attn in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_attn in layer 1/12...
INFO - Quantizing attn.c_proj in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_proj in layer 1/12...
INFO - Quantizing mlp.c_fc in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_fc in layer 1/12...
INFO - Quantizing mlp.c_proj in layer 1/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_proj in layer 1/12...
INFO - Start quantizing layer 2/12
INFO:auto_gptq.modeling._base:Start quantizing layer 2/12
INFO - Quantizing attn.c_attn in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_attn in layer 2/12...
INFO - Quantizing attn.c_proj in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing attn.c_proj in layer 2/12...
INFO - Quantizing mlp.c_fc in layer 2/12...
INFO:auto_gptq.modeling._base:Quantizing mlp.c_fc in layer 2/12...
INFO - Qu

('gpt2-GPTQ/tokenizer_config.json',
 'gpt2-GPTQ/special_tokens_map.json',
 'gpt2-GPTQ/vocab.json',
 'gpt2-GPTQ/merges.txt',
 'gpt2-GPTQ/added_tokens.json',
 'gpt2-GPTQ/tokenizer.json')

## Load quantized model for inference

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Reload model and tokenizer
model = AutoGPTQForCausalLM.from_quantized(
    out_dir,
    device=device,
    use_triton=True,
    use_safetensors=True,
)
tokenizer = AutoTokenizer.from_pretrained(out_dir)

WARNING - ignoring unknown parameter in quantize_config.json: quant_method.
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


## Inference

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = generator("I have a dream", do_sample=True, max_length=50)[0]['generated_text']
print(result)

The model 'GPT2GPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFor

I have a dream for the only that it it I, and a it it in, it it she


(To help up all her

in and for a little, (in a, you can a c the it d,


# Pre quantized Model from HF- Mistral-7B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Write a story about llamas"
system_message = "You are a story writing assistant"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# Another Method to quantize using GPTQ via Optimum library

## Install Libraries

## Load Libraries

In [ ]:
import torch
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model

## Load and Quantize Model

### OPT-125m

In [ ]:

model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

quantizer = GPTQQuantizer(bits=4, dataset="c4", block_name_to_quantize = "model.decoder.layers", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4373: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


### InternLM-7b

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch
model_name = "internlm/internlm2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16,trust_remote_code=True) #trust_remote_code=True

quantizer = GPTQQuantizer(bits=4, dataset="c4", model_seqlen = 2048) #block_name_to_quantize = "model.decoder.layers"
quantized_model = quantizer.quantize_model(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

The repository for internlm/internlm2-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/internlm/internlm2-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_internlm2_fast.py:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

tokenization_internlm2.py:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- tokenization_internlm2_fast.py
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Quantizing layers inside the block:   0%|          | 0/5 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/5 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/5 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/5 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/5 [00:00<?, ?it/s]

## **Save the quantized model**

In [ ]:
out_dir = "quantized-internlm7b-gptq"
quantized_model.save_pretrained(out_dir, use_safetensors=True)
tokenizer.save_pretrained(out_dir)

('quantized-internlm7b-gptq/tokenizer_config.json',
 'quantized-internlm7b-gptq/special_tokens_map.json',
 'quantized-internlm7b-gptq/./tokenizer.model',
 'quantized-internlm7b-gptq/added_tokens.json',
 'quantized-internlm7b-gptq/tokenizer.json')

In [ ]:
from huggingface_hub import login
login()

### Push to hub

In [ ]:
quantized_model.push_to_hub("steve1989/internLM-7b-gptq-4bit")

model-00002-of-00002.safetensors:   0%|          | 0.00/758M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/steve1989/internLM-7b-gptq-4bit/commit/8c166f818a7d7e247d7e1e049c5b23cff7289518', commit_message='Upload InternLM2ForCausalLM', commit_description='', oid='8c166f818a7d7e247d7e1e049c5b23cff7289518', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("steve1989/internLM-7b-gptq-4bit")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/steve1989/internLM-7b-gptq-4bit/commit/6aade14a1729a0a8111d3ec66738d922ee7ef28f', commit_message='Upload tokenizer', commit_description='', oid='6aade14a1729a0a8111d3ec66738d922ee7ef28f', pr_url=None, pr_revision=None, pr_num=None)

## **Load the quantized model**

In [ ]:
model_name

'internlm/internlm2-7b'

In [ ]:
from accelerate import init_empty_weights
with init_empty_weights():
    empty_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16,trust_remote_code=True)
empty_model.tie_weights()
quantized_model = load_quantized_model(empty_model, save_folder=out_dir, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: gpt2-GPTQ is not a folder containing a `.index.json` file or a pytorch_model.bin or a model.safetensors file

## Evaluation - Financial Phrasebank

### Load Dataset - Imbalanced

In [ ]:
from datasets import load_dataset

fpb_dataset = load_dataset("financial_phrasebank",'sentences_allagree')

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

###Format the dataset as a prompt

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# tokenizer.to(device)

In [ ]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(fpb_dataset['train']['sentence'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fpb_dataset['train']['sentence'][i]}. Answer:")

fpb_dataset['train'] = fpb_dataset['train'].add_column("prompt_2", p)

In [ ]:
fpb_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'prompt_2'],
        num_rows: 2264
    })
})

### Tokenize

In [ ]:
# tokenized_set_1 = flant5_tokenizer(prompt, return_tensors='pt', padding=True,max_length=512)

#---------------------------------------------------------
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
tokenized_set_gptq = tokenizer.batch_encode_plus(
    fpb_dataset['train']['prompt_2'], #prompt
    return_tensors='pt',
    add_special_tokens=True,
    return_attention_mask=True,
    # pad_to_max_length=True,  Deprecated
    padding='max_length',
    truncation=True,
    max_length=175 # the maximum lenght observed in the headlines, as can be seen in the above distribution
)

In [ ]:
input_ids_val = tokenized_set_gptq['input_ids']
attention_masks_val = tokenized_set_gptq['attention_mask']

dataset_val = TensorDataset(input_ids_val, attention_masks_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=5)

In [ ]:
import time, torch
predictions_gptq = []
start_time = time.time()
model.eval()

for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200)
    # outputs = flant5_model(**inputs)

  res_sentences = [tokenizer.decode(i) for i in outputs]
  # out_text = [o.split("Answer:")[1] for o in res_sentences]

  # # show results
  # for gen_text in out_text:
  #     print(gen_text)

  end_time = time.time()
  execution_time = end_time - start_time

  predictions_gptq.append(res_sentences)
# print("Result:", res_sentences)
print("Execution time:", execution_time, "seconds")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

###**Extract response from models output**

In [ ]:
predictions_gptq

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
final_sent_answer_gptq=[]
for batch in predictions_gptq:
  for sent_output in batch:
    final_sent_answer_gptq.append(sent_output.split('Answer:')[1].split('<')[0])

In [ ]:
for index,res in enumerate(final_sent_answer_gptq):
  final_sent_answer_gptq[index] = res.replace(" ","")

In [ ]:
len(final_sent_answer_gptq)

2264

In [ ]:
final_sent_answer_gptq[100:120]

['Positive',
 'Positive',
 'Positive',
 'Positive',
 'positive',
 'neutral',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Positive']

In [ ]:
# prompt: convert list to dataframe with column called 'response'

import pandas as pd
df_res = pd.DataFrame(final_sent_answer_gptq, columns=['response'])


In [ ]:
df_res['label'] = [1] *2264

In [ ]:
df_res.tail(5)

,response,label
2259,Negative,0
2260,Negative,0
2261,Negative,0
2262,Negative,0
2263,neutral,1


In [ ]:
for index,res in enumerate(df_res['response']):
  # print(index)
  if df_res['response'][index].lower() == 'positive':
    df_res['label'][index] = 2
  elif df_res['response'][index].lower() == 'negative':
    df_res['label'][index] = 0
  elif df_res['response'][index].lower() == 'neutral':
    df_res['label'][index] = 1


In [ ]:
import pandas as pd
df_res.to_csv('final_sent_answer_Internlm-7b_gptq.csv')

### Accuracy

In [ ]:
# prompt: code to convert dataset object fpb_dataset to pandas dataframe object

fpb_dataset_pandas = fpb_dataset['train'].to_pandas()
# Compare the values in the two columns
comparison = fpb_dataset_pandas["label"] == df_res["label"]

# Calculate the accuracy
accuracy = comparison.sum() / len(comparison)

# Print the accuracy
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.86


### Precision

In [ ]:
# prompt: code to calculate precision

from sklearn.metrics import precision_score

y_true = fpb_dataset['train']["label"]
y_pred = df_res["label"]

precision = precision_score(y_true, y_pred, average='macro')
print("Macro-averaged precision:", precision)

Macro-averaged precision: 0.8735188323917137


### F1 score

In [ ]:
# prompt: code to calculate f1 score

from sklearn.metrics import f1_score

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='macro')

# Print F1 score
print("Macro-averaged F1 score:", f1)


Macro-averaged F1 score: 0.8343968402317943


### Recall

In [ ]:
# prompt: code to calculate recall

from sklearn.metrics import recall_score

# Calculate recall score
recall = recall_score(y_true, y_pred, average='macro')

# Print recall score
print("Macro-averaged recall:", recall)


Macro-averaged recall: 0.8056149816427025


##**Evaluate - Financial News headlines**

### Load Model

#### Load from Hugging face

In [ ]:
model_id ="steve1989/internLM-7b-gptq-4bit"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("steve1989/internLM-7b-gptq-4bit")
config.quantization_config["disable_exllama"] = True
# Reload model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #device=device,
    config=config,
    use_safetensors=True,trust_remote_code=True,revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

The repository for steve1989/internLM-7b-gptq-4bit contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/steve1989/internLM-7b-gptq-4bit.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_internlm2.py:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/steve1989/internLM-7b-gptq-4bit:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internlm2.py:   0%|          | 0.00/60.0k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

KeyboardInterrupt: 

#### Load from directory

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Reload model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    out_dir,
    device=device,
    use_triton=True,
    use_safetensors=True,trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(out_dir)

#### Third way to load

In [ ]:
from optimum.gptq import GPTQQuantizer, load_quantized_model
model_name = "internlm/internlm2-7b"
from accelerate import init_empty_weights
with init_empty_weights():
    empty_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
empty_model.tie_weights()
quantized_model = load_quantized_model(empty_model, save_folder=out_dir, device_map="auto")

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

The repository for internlm/internlm2-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/internlm/internlm2-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_internlm2.py:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for internlm/internlm2-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/internlm/internlm2-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

ValueError: gpt2-GPTQ is not a folder containing a `.index.json` file or a pytorch_model.bin or a model.safetensors file

### Load Dataset - balanced

In [ ]:
dataset_id = "steve1989/financial_news_headlines"
fin_headlines_dataset = load_dataset("steve1989/financial_news_headlines")

###Format the dataset as a prompt

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# tokenizer.to(device)

In [ ]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(fin_headlines_dataset['test']['Headlines'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fin_headlines_dataset['test']['Headlines'][i]}. Answer:")

fin_headlines_dataset['test'] = fin_headlines_dataset['test'].add_column("prompt_2", p)

In [ ]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(fin_headlines_dataset['train']['Headlines'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fin_headlines_dataset['train']['Headlines'][i]}. Answer:")
fin_headlines_dataset['train'] = fin_headlines_dataset['train'].add_column("prompt_2", p)

labels = "positive/negative/neutral"
p=[]
for i in range(len(fin_headlines_dataset['validation']['Headlines'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fin_headlines_dataset['validation']['Headlines'][i]}. Answer:")
fin_headlines_dataset['validation'] = fin_headlines_dataset['validation'].add_column("prompt_2", p)

In [ ]:
fin_headlines_dataset

DatasetDict({
    train: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2'],
        num_rows: 13031
    })
    validation: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2'],
        num_rows: 2793
    })
    test: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2'],
        num_rows: 2793
    })
})

In [ ]:
fin_headlines_dataset['test']['sentiment_label'][0]

'Neutral'

In [ ]:
def add_label(example):
  if example['sentiment_label'].lower() == 'positive':
    example['label'] = 2
  elif example['sentiment_label'].lower() == 'negative':
    example['label'] = 0
  elif example['sentiment_label'].lower() == 'neutral':
    example['label'] = 1
  return {'label': example['label']}

fin_headlines_dataset['train'] = fin_headlines_dataset['train'].map(add_label)
fin_headlines_dataset['validation'] = fin_headlines_dataset['validation'].map(add_label)
fin_headlines_dataset['test'] = fin_headlines_dataset['test'].map(add_label)

Map:   0%|          | 0/13031 [00:00<?, ? examples/s]

Map:   0%|          | 0/2793 [00:00<?, ? examples/s]

Map:   0%|          | 0/2793 [00:00<?, ? examples/s]

In [ ]:
fin_headlines_dataset

DatasetDict({
    train: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2', 'label'],
        num_rows: 13031
    })
    validation: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2', 'label'],
        num_rows: 2793
    })
    test: Dataset({
        features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2', 'label'],
        num_rows: 2793
    })
})

### Tokenize

In [ ]:
# tokenized_set_1 = flant5_tokenizer(prompt, return_tensors='pt', padding=True,max_length=512)

#---------------------------------------------------------

tokenized_set_gptq = tokenizer.batch_encode_plus(
    fin_headlines_dataset['test']['prompt_2'], #prompt
    return_tensors='pt',
    add_special_tokens=True,
    return_attention_mask=True,
    # pad_to_max_length=True,  Deprecated
    padding=True,
    truncation=True,
    max_length=175 # the maximum lenght observed in the headlines, as can be seen in the above distribution
)

In [ ]:
input_ids_val = tokenized_set_gptq['input_ids']
attention_masks_val = tokenized_set_gptq['attention_mask']

dataset_val = TensorDataset(input_ids_val, attention_masks_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=5)

In [ ]:
import time, torch
predictions_gptq = []
start_time = time.time()
model.eval()

for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200)
    # outputs = flant5_model(**inputs)

  res_sentences = [tokenizer.decode(i) for i in outputs]
  # out_text = [o.split("Answer:")[1] for o in res_sentences]

  # # show results
  # for gen_text in out_text:
  #     print(gen_text)

  end_time = time.time()
  execution_time = end_time - start_time

  predictions_gptq.append(res_sentences)
# print("Result:", res_sentences)
print("Execution time:", execution_time, "seconds")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

###**Extract response from models output**

In [ ]:
final_gptq_preds = []
for batch in predictions_gptq:
  for item in batch:
    first_split = item.split('Answer:')[1]
    if 'neutral' in first_split.lower():
      final_gptq_preds.append('neutral')
    elif 'positive' in first_split.lower():
      final_gptq_preds.append('positive')
    elif 'negative' in first_split.lower():
      final_gptq_preds.append('negative')
    else:
      final_gptq_preds.append('na')

In [ ]:
len(final_gptq_preds)

2793

In [ ]:
# final_gptq_preds[:100]

In [ ]:
final_pred_label = []
for pred in final_gptq_preds:
  if pred.lower() == 'positive':
    final_pred_label.append(2)
  elif pred.lower() == 'negative':
    final_pred_label.append(0)
  elif pred.lower() == 'neutral':
    final_pred_label.append(1)

In [ ]:
len(final_pred_label)

2793

### Calculate Metrics - On test set

In [ ]:
import evaluate, numpy as np
headlines_test_predictions = np.array(final_pred_label)
headlines_true_vals = np.array(fin_headlines_dataset['test']['label'])

#### Accuracy on Test set

In [ ]:

# ---------------------------------OR------------------------------------------------
accuracy_metric = evaluate.load("accuracy")
acc_results = accuracy_metric.compute(references=headlines_true_vals, predictions=headlines_test_predictions) #
print(acc_results)


{'accuracy': 0.5639097744360902}


#### Precision on Test set

In [ ]:

# ---------------------------------OR------------------------------------------------
pre_metric = evaluate.load("precision")
pre_results = pre_metric.compute(references=headlines_true_vals, predictions=headlines_test_predictions,average='micro') #
print(pre_results)


{'precision': 0.5639097744360902}


####Recall on Test set

In [ ]:

# ---------------------------------OR------------------------------------------------
r_metric = evaluate.load("recall")
r_results = r_metric.compute(references=headlines_true_vals, predictions=headlines_test_predictions,average='micro') #
print(r_results)


{'recall': 0.5639097744360902}


#### F1 score on test set

In [ ]:

# ---------------------------------OR------------------------------------------------
f1_metric = evaluate.load("f1")
f1_results = f1_metric.compute(references=headlines_true_vals, predictions=headlines_test_predictions,average='micro') #
print(f1_results)


{'f1': 0.5639097744360902}


##Fine tuning on Financial news headlines

### Find Max length of input and target tokens

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([fin_headlines_dataset["train"], fin_headlines_dataset["validation"]]).map(lambda x: tokenizer(x["Headlines"], truncation=True), batched=True, remove_columns=['Headlines', 'sentiment_label', '__index_level_0__','label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([fin_headlines_dataset["train"], fin_headlines_dataset["validation"]]).map(lambda x: tokenizer(x["sentiment_label"], truncation=True), batched=True, remove_columns=['Headlines', 'sentiment_label', '__index_level_0__','label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/15824 [00:00<?, ? examples/s]

Max source length: 64


Map:   0%|          | 0/15824 [00:00<?, ? examples/s]

Max target length: 2


### Tokenize the dataset.

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    prefix ="What is the sentiment of this news? Please choose an answer from {positive/negative/neutral} News: "

    inputs = [prefix + item for item in sample["Headlines"]]
    final_inputs = [i + " .Asnwer: " for i in inputs]
    # tokenize inputs
    model_inputs = tokenizer(final_inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["sentiment_label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = fin_headlines_dataset.map(preprocess_function, batched=True, remove_columns=['Headlines', 'sentiment_label', '__index_level_0__','label','prompt_2'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/13031 [00:00<?, ? examples/s]

Map:   0%|          | 0/2793 [00:00<?, ? examples/s]

Map:   0%|          | 0/2793 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13031
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2793
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2793
    })
})

**Retrieve the target modules.**

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)
#
modules = find_all_linear_names(model)
print(modules)

[]


In [ ]:
model

GPT2GPTQForCausalLM(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): QuantLinear()
            (c_proj): QuantLinear()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): QuantLinear()
            (c_proj): QuantLinear()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
internlm7b_model_ = prepare_model_for_kbit_training(model)
#
print(internlm7b_model_)

#

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["lm_head","wte","wpe",],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

internlm7b_peft_model = get_peft_model(internlm7b_model_, lora_config)

GPT2GPTQForCausalLM(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): QuantLinear()
            (c_proj): QuantLinear()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): QuantLinear()
            (c_proj): QuantLinear()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)


**List the number of trainable parameters.**

In [ ]:
trainable, total = internlm7b_peft_model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 830736 | total: 40252944 | Percentage: 2.0638%


###Compute Metrics

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    # result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


SyntaxError: invalid decimal literal (rouge.py, line 30)

In [ ]:
from transformers import DataCollatorForLanguageModeling

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    # model=internlm7b_peft_model,
    # label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,mlm=False
)


### **Initiate the training.**

In [ ]:
import transformers

# from trl import SFTTrainer
# repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

tokenizer.pad_token = tokenizer.eos_token
# internlm7b_tokenizer.padding_side='right'
torch.cuda.empty_cache()

# def train(config=None):
#   with wandb.init(config=config):
#     # set sweep configuration
#     config = wandb.config
training_args = transformers.TrainingArguments(
    gradient_accumulation_steps=4,
    output_dir="GPTQ-financial-headlines",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True, # Overflows with fp16
    learning_rate=5e-5,
    # num_train_epochs=5,
    max_steps=100,
    # logging & evaluation strategies
    logging_dir="GPTQ-financial-headlines/logs",
    logging_strategy="steps",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
    optim="paged_adamw_8bit",
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    # torch_compile=True
    # hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)
internlm_trainer = transformers.Trainer(
        model=internlm7b_peft_model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['validation'],
        # compute_metrics=compute_metrics,
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),)
    #
internlm7b_peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# torch.cuda.empty_cache()


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# wandb.agent(sweep_id, train, count=20)

### Trainer

In [ ]:
internlm_trainer.train()

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['labels']